In [79]:
%load_ext autoreload

%autoreload 2

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 - create a SBM graph and extract the edges list


In [80]:
size = 100
s = size
sizes = [s, s, s]
probs = np.array([[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]])/10
g = nx.stochastic_block_model(sizes, probs, seed=0)
'''
pin = .05
pout = .1
probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]
#probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]

g = nx.stochastic_block_model(sizes, probs, seed=0, directed=True)
'''
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])
edges_list = np.array(edges_list)


In [81]:
#plt.imshow(nx.adjacency_matrix(g).todense())

### node/edge colors and shape

In [82]:
# update edge colors
colors_by_block = [[1, 0, 0], [0, 1, 0,], [0, 0, 1]]
#colors_by_block = [[1, 0, 0, 240], [0, 1, 0, 240], [0, 0, 1, 0]]

edge_colors = []
for source, target in g.edges():
    c0 = colors_by_block[source//s]
    c1 = colors_by_block[target//s]
    edge_colors += [c0, c1]
    
colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array(
    [colors_by_block[i//s] 
     for i in range(len(g))]).astype('float64')
markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]

edge_colors = np.array(edge_colors).astype('float64')

### 2 - Creates a network superactor and adds to a FURY Scene

In [110]:
from helios.fury.actors import FurySuperActorNetwork
from fury import window



    
centers = np.random.normal(size=(len(g), 3))


network_super_actor = FurySuperActorNetwork(
        positions=centers,
        colors=colors,
        scales=1,
        node_edge_width=np.random.uniform(size=len(g)),
        marker=markers,
        edge_line_color=edge_colors,
        edge_line_width=1,
        edges=edges_list
)

nodes = network_super_actor.nodes.vtk_actor

scene = window.Scene()
scene.add(nodes)
scene.add(network_super_actor.edges.vtk_actor)

In [111]:
network_super_actor.nodes._edge_width

array([0.37674289, 0.37674289, 0.37674289, ..., 0.99839404, 0.99839404,
       0.99839404])

In [112]:
#print(dir(network_super_actor.edges.vtk_actor.GetProperty()))

### 3 - Creates a ShowManager FURY Obj.

In [113]:
from fury import actor, window

window_size = [800, 600]
showm = window.ShowManager(scene, reset_camera=False, size=(
    window_size[0], window_size[1]), order_transparent=False,
)

### 4- Starts the streaming

In [114]:
from fury.stream.widget import Widget
#widget = Widget(showm, encoding='webrtc', port=8777)
widget = Widget(showm)
widget.start()

url: http://localhost:8153?iframe=1&encoding=mjpeg


In [115]:
scene.background((.0, .0, .0))
showm.window.Render()

In [116]:
widget.return_iframe(300)

### 5 - Start Force-Directed async layout

- ms equals to 100 means that the vtk_positions will be updated at each 100 ms

In [117]:
from helios.core.layout import HeliosFr
layout = HeliosFr(centers,edges_list,showm.window, network_super_actor, update_interval_workers=0)

start async helios


In [119]:
layout.steps(1000)

In [ ]:
layout.start(50)

In [91]:
layout.stop()

### 6 - Change some network properties

In [122]:
network_super_actor.nodes.edge_width = np.ones(len(g))/2
showm.window.Render()

In [32]:
network_super_actor.nodes.marker_opacity = .5
# if the network actor edge_width was an uniform variable
#network_super_actor.nodes.edge_width = .1
network_super_actor.nodes.Uniforms.edgeColor.value = (255, 255, 255)
showm.window.Render()

In [45]:
network_super_actor.edges.blending='additive'
#network_super_actor.edges.vtk_actor.GetProperty().SetOpacity(.5)
#network_super_actor.edges.vtk_actor.GetProperty().ShadingOn()
network_super_actor.edges.start_effects(showm.window)
showm.window.Render()

In [11]:
network_super_actor.edges.blending='subtractive'
network_super_actor.edges.start_effects(showm.window)
showm.window.Render()

In [46]:
network_super_actor.nodes.blending='additive'
network_super_actor.nodes.start_effects(showm.window)
showm.window.Render()

In [39]:
scene.background((0, 0, 0))
showm.window.Render()

In [48]:
layout.stop()

In [49]:
widget.stop()